# How to calculate Risk
We also created a function used to estimate the risk of a certain strategic voting an agent might consider to use instead of its honest preferences.

***

## Preparations

In [17]:
import numpy as np

Happiness function:

In [19]:
# i = outcome; j = voted; s = discount

def func(i: np.array, j: np.array, s: float=0.9) -> float:
    m = len(i)
    s1 = np.power([s for l in range(m)], np.power(range(m), 2))
    s2 = np.power([s for l in range(m)], np.power(m - np.array(range(m)), 2))
    d = np.abs(j - i)
    h_hat = np.sum(d * (s1 + s2)) / m
    return np.exp(-h_hat)

Functions needed to use the happiness function:

In [29]:
def translate_to_int_array(ranking, mapping):
    numbers = []
    for element in ranking:
        index = 1
        for m in mapping:
            if(element == m):
                numbers.append(index)
                break
            index += 1
    return np.asarray(numbers)

def translate_to_content_array(ranking, mapping):
    values = []
    for element in ranking:
        values.append(mapping[element-1])
    return np.asarray(values)

In [31]:
vote = np.array(["A","B","C","D","E"])
outcome = np.array([3,2,1,4,5])
vote_numbers = translate_to_int_array(vote,vote)
outcome_content = translate_to_content_array(outcome,vote)

print("vote (values): ", vote)
print("vote (int): ", vote_numbers)
print("outcome (int): ", outcome)
print("outcome (values): ", outcome_content)

vote (values):  ['A' 'B' 'C' 'D' 'E']
vote (int):  [1 2 3 4 5]
outcome (int):  [3 2 1 4 5]
outcome (values):  ['C' 'B' 'A' 'D' 'E']


***

## Example

In [8]:
honest_vote = np.array(["A","B","C","D","E"])
estimated_opponent_votes = np.array([
    ["B","A","C","D","E"],
    ["C","B","A","D","E"],
    ["D","B","C","A","E"],
    ["E","B","C","D","A"]
])

***

## Functions

### 1. calculate the result of a vote

Method:

In [11]:
def get_result (votings):
    cs = []
    for candidate in votings[0]:
        cs.append(str(candidate))
    candidates = np.array(cs)
    countings = np.zeros(len(votings[0]))
    for vote in votings:
        score = len(votings[0])
        for rank in vote:
            index = 0
            for candidate in candidates:
                if(rank == candidate):
                    countings[index] += score
                    break
                index += 1
            score -= 1
    result = list(zip(candidates,countings))
    result.sort(key=lambda x:x[1], reverse = True)
    return [x for (x,y) in result]

Test:

In [12]:
all_votes = np.concatenate((estimated_opponent_votes,honest_vote.reshape((1,5))),axis=0)
estimated_honest_result = get_result(all_votes)
print(all_votes)
print(estimated_honest_result)

[['B' 'A' 'C' 'D' 'E']
 ['C' 'B' 'A' 'D' 'E']
 ['D' 'B' 'C' 'A' 'E']
 ['E' 'B' 'C' 'D' 'A']
 ['A' 'B' 'C' 'D' 'E']]
['B', 'C', 'A', 'D', 'E']


### 2. calculate a strategic vote

Method:

Test:

In [13]:
strategic_vote = np.array(["B","A","C","D","E"])

### 3. calculate the risk of a strategic vote

Approach:

1. determine loss of happiness if strategic voting became result
2. multiply this loss with the number of assumptions under which the strategy makes sense

Method:

In [32]:
def get_risk (strategic_vote : np.array, honest_vote : np.array, estimated_honest_outcome : np.array, estimated_opponent_votes : np.array):
    #estimated_strategic_outcome = get_result(np.concatenate((estimated_opponent_votes,honest_vote.reshape((1,5))),axis=0))
    #honest_happiness = func(estimated_honest_outcome,honest_vote)
    return func(translate_to_int_array(strategic_vote,strategic_vote),translate_to_int_array(honest_vote,strategic_vote),0.9)

Test:

In [33]:
print(strategic_vote)
print(honest_vote)
print(get_risk(strategic_vote,honest_vote,estimated_honest_result,estimated_opponent_votes))

['B' 'A' 'C' 'D' 'E']
['A' 'B' 'C' 'D' 'E']
0.6495870903701048
